# Z-Score Normalization

[]

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split

DATA_PATH = './data'
ratings = pd.read_csv(f'{DATA_PATH}/ratings.csv')
movies  = pd.read_csv(f'{DATA_PATH}/movies.csv')

#SEED_LIST = [10, 21, 35, 42, 57, 60, 73, 88, 95, 101] -> 5개로 감소
SEED_LIST = [10, 21, 35, 42, 57]
PERSONA_HEAVY_USER = 414
PERSONA_GENRE_SPECIALIST = 85

## 1. 전처리 함수 (Z-Score + 최소 평점 필터링)

In [2]:
def run_preprocessing(train_df, test_df):
    train = train_df.copy()
    test  = test_df.copy()

    # 1. 최소 5회 이상 평점 준 사용자만 사용 (페르소나는 강제 포함)
    user_counts = train['userId'].value_counts()
    valid_users = set(user_counts[user_counts >= 5].index)
    valid_users = valid_users.union({PERSONA_HEAVY_USER, PERSONA_GENRE_SPECIALIST})

    train = train[train['userId'].isin(valid_users)]
    test  = test[test['userId'].isin(valid_users)]

    # 2. Train에서만 사용자별 mean / std 계산
    user_stats = train.groupby('userId')['rating'].agg(['mean', 'std']).fillna(1.0)
    user_stats['std'] = user_stats['std'].replace(0, 1.0)   # std=0 방지

    # 3. Z-Score 변환 (Train)
    train = train.merge(user_stats, on='userId', suffixes=('', '_user'))
    train['z_rating'] = (train['rating'] - train['mean']) / train['std']

    # 4. Test도 동일한 통계량으로 변환 (Train에 없는 사용자는 제거 → cold-start 방지)
    test = test.merge(user_stats, on='userId', how='left')
    test = test.dropna(subset=['mean', 'std'])   # Train에 없는 사용자 제거
    test['z_rating'] = (test['rating'] - test['mean']) / test['std']

    return train, test, user_stats   # user_stats도 반환 (역변환에 필요)

## 2. 평가 함수

In [3]:
import numpy as np
import time

def fast_predict_matrix_no_k(train_df):
    """
    K 제한 없이, 유사도가 양수인 모든 아이템을 사용하여 예측
    """
    # 1. Pivot Table 생성 및 R 행렬 정의
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values  # (Users, Items)
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns
    
    # 2. 아이템 간 유사도 행렬 계산
    sim_matrix = np.dot(R.T, R)
    np.fill_diagonal(sim_matrix, -np.inf)
    
    # 3. [핵심 변경] Top-K 필터링 로직 삭제! 양수 유사도만 남김
    S = np.where(sim_matrix > 0, sim_matrix, 0)

    # 4. 예측 평점 계산 (Weighted Sum) -> 분자
    numerator = np.dot(R, S)
    
    # 5. 가중치 합 계산 (Sum of Weights) -> 분모
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    # 6. 나눗셈 (0으로 나누기 방지)
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        pred_z = np.nan_to_num(pred_z)
        
    return pred_z, user_ids, movie_ids

In [4]:
def rmse(y_true, y_pred): return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))
def mae(y_true, y_pred):  return np.mean(np.abs(np.array(y_true)-np.array(y_pred)))

def time_predictions(fn, pairs):
    s = time.perf_counter()
    preds = [fn(u,i) for u,i in pairs]
    e = time.perf_counter() - s
    return preds, e, (e/len(pairs))*1000

추가 1

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
import time

def calculate_genre_similarity(movies_df):
    """
    movies DataFrame을 사용하여 영화 간 장르 코사인 유사도 행렬을 계산합니다.
    """
    # 1. 장르 컬럼 분리 및 One-Hot Encoding
    # .copy()를 사용하여 원본 DataFrame 변경 방지
    movies_df = movies_df.copy()
    movies_df['genres_list'] = movies_df['genres'].apply(lambda x: x.split('|'))
    
    mlb = MultiLabelBinarizer()
    genre_matrix = pd.DataFrame(mlb.fit_transform(movies_df['genres_list']),
                                columns=mlb.classes_,
                                index=movies_df['movieId'])
    
    # 'no genres listed'는 제거
    if '(no genres listed)' in genre_matrix.columns:
        genre_matrix = genre_matrix.drop(columns=['(no genres listed)'])

    # 2. 코사인 유사도 계산
    # Scikit-learn의 cosine_similarity를 사용하여 정규화까지 한 번에 처리
    S_genre_array = cosine_similarity(genre_matrix.values)
    
    S_genre_df = pd.DataFrame(S_genre_array, 
                              index=genre_matrix.index, 
                              columns=genre_matrix.index)
    
    # 자기 자신과의 유사도는 0으로 설정
    np.fill_diagonal(S_genre_df.values, 0.0)
    
    return S_genre_df

def fast_predict_matrix_hybrid(train_df, S_genre_df, alpha=0.7):
    """
    하이브리드 유사도 행렬을 사용하여 예측 평점을 계산합니다.
    S_hybrid = alpha * S_rating + (1 - alpha) * S_genre
    """
    # 1. Pivot Table 생성 및 R 행렬 정의
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns
    
    # 2. 평점 기반 유사도 행렬 계산 (S_rating)
    sim_matrix_rating = np.dot(R.T, R)
    
    # 3. [핵심 변경] S_rating과 S_genre를 결합
    
    # 3-1. S_genre를 S_rating의 인덱스/컬럼 크기에 맞게 자름
    S_genre_aligned = S_genre_df.loc[movie_ids, movie_ids].values

    # 3-2. Hybrid 유사도 계산
    # S_rating을 양수/음수 모두 사용하되, 대각선은 0으로 처리
    sim_matrix_rating_clean = sim_matrix_rating.copy()
    np.fill_diagonal(sim_matrix_rating_clean, 0.0)

    # Hybrid 유사도 행렬 S_hybrid 생성
    S_hybrid = alpha * sim_matrix_rating_clean + (1 - alpha) * S_genre_aligned
    
    # 4. Hybrid 유사도 행렬을 최종 S로 정의
    S = S_hybrid
    
    # 5. 예측 평점 계산 (Weighted Sum) -> 분자
    numerator = np.dot(R, S)
    
    # 6. 가중치 합 계산 (Sum of Weights) -> 분모
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    # 7. 나눗셈
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        pred_z = np.nan_to_num(pred_z)
        
    return pred_z, user_ids, movie_ids

## 3. Multi-seed 실험 루프

In [20]:
# ==========================================
# [3. Multi-seed 실험 루프 - 하이브리드 & Clamping 적용 버전]
# 기존 셀 내용을 모두 대체합니다.
# ==========================================

results = []
best_reco = None

# 1. [초기화] 장르 유사도 행렬을 한 번만 계산
S_genre_df = calculate_genre_similarity(movies.copy()) 
ALPHA = 0.3
print(f"하이브리드 실험 시작: 평점 {ALPHA*100:.0f}%, 장르 {(1-ALPHA)*100:.0f}% 반영. 총 {len(SEED_LIST)}개 시드 테스트.")

for idx, SEED in enumerate(SEED_LIST):
    print(f"\nRunning Seed {SEED} ({idx+1}/{len(SEED_LIST)})")

    # 2. 데이터 분할 및 전처리
    train_raw, test_raw = train_test_split(ratings, test_size=0.2, random_state=SEED)
    train, test, user_stats = run_preprocessing(train_raw, test_raw)

    # 3. [핵심] 하이브리드 행렬 연산으로 모든 예측값 한 번에 계산
    s = time.perf_counter()
    pred_matrix_z, u_ids, m_ids = fast_predict_matrix_hybrid(train, S_genre_df, alpha=ALPHA) 
    e = time.perf_counter() - s
    
    # 인덱싱 맵핑
    u_map = {u: i for i, u in enumerate(u_ids)}
    m_map = {m: i for i, m in enumerate(m_ids)}

    # 4. Test 샘플링 및 예측값 추출
    rng = np.random.default_rng(SEED)
    test_sample = test[['userId', 'movieId', 'rating', 'mean', 'std']].sample(n=min(3000, len(test)), random_state=SEED)

    z_preds = []
    for uid, mid in zip(test_sample['userId'], test_sample['movieId']):
        if uid in u_map and mid in m_map:
            u_idx = u_map[uid]
            m_idx = m_map[mid]
            z_preds.append(pred_matrix_z[u_idx, m_idx])
        else:
            z_preds.append(0.0)

    # 5. 역변환 및 결과 저장
    # Test 샘플의 예측값은 RMSE/MAE 계산에 사용되므로 clamping이 필요 없으나,
    # 예측값 스케일이 5.0을 넘지 않도록 여기서도 클리핑을 적용하는 것이 좋습니다.
    original_preds = []
    for pred_z, row in zip(z_preds, test_sample.itertuples()):
        mean_u = row.mean
        std_u  = row.std
        raw_score = pred_z * std_u + mean_u
        # [수정] RMSE/MAE 계산을 위한 예측값도 클리핑 (0.5 ~ 5.0)
        final_score = np.clip(raw_score, 0.5, 5.0)
        original_preds.append(final_score)


    avg_ms = (e / len(test_sample)) * 1000 
    
    res = {
        'seed': SEED,
        'rmse': rmse(test_sample['rating'], original_preds),
        'mae' : mae(test_sample['rating'], original_preds),
        'avg_ms': avg_ms
    }
    results.append(res)
    print(f" -> RMSE: {res['rmse']:.4f}, Time: {e:.4f}s")

    # 6. 첫 번째 시드(Seed 10)에서만 추천 결과 생성
    if idx == 0:
        def recommend(user_id, topk=3):
            if user_id not in u_map: return None
            u_idx = u_map[user_id]
            user_row_preds = pred_matrix_z[u_idx] 
            watched = set(train[train['userId']==user_id]['movieId'])
            
            candidates = []
            for m_idx, m_id in enumerate(m_ids):
                if m_id in watched: continue
                
                z = user_row_preds[m_idx]
                mean_u = user_stats.loc[user_id, 'mean']
                std_u  = user_stats.loc[user_id, 'std']
                
                # [핵심 수정] 예측 평점 클리핑 (Clamping) 적용
                raw_score = z * std_u + mean_u
                final_score = np.clip(raw_score, 0.5, 5.0) 
                
                candidates.append((m_id, final_score))
            
            candidates.sort(key=lambda x: -x[1])
            top = candidates[:topk]
            
            return [{
                'movieId': m,
                'title': movies.loc[movies['movieId']==m, 'title'].iloc[0] if len(movies.loc[movies['movieId']==m]) > 0 else "Unknown",
                'predicted_rating': round(score, 3)
            } for m, score in top]

        print("\n[추천 결과 생성 중...]")
        best_reco = {
            'heavy': recommend(PERSONA_HEAVY_USER),
            'specialist': recommend(PERSONA_GENRE_SPECIALIST)
        }

print("\n실험 완료! (결과 요약 셀을 실행하세요)")

하이브리드 실험 시작: 평점 30%, 장르 70% 반영. 총 5개 시드 테스트.

Running Seed 10 (1/5)
 -> RMSE: 0.8492, Time: 15.7344s

[추천 결과 생성 중...]

Running Seed 21 (2/5)
 -> RMSE: 0.8651, Time: 9.1257s

Running Seed 35 (3/5)
 -> RMSE: 0.8629, Time: 10.9691s

Running Seed 42 (4/5)
 -> RMSE: 0.8781, Time: 7.2759s

Running Seed 57 (5/5)
 -> RMSE: 0.8392, Time: 6.9810s

실험 완료! (결과 요약 셀을 실행하세요)


## 4. 결과 요약

In [21]:
import pandas as pd
df = pd.DataFrame(results)
print(df[['seed','rmse','mae','avg_ms']].round(4))
print("\n=== 평균 ± 표준편차 ===")
print(f"RMSE : {df.rmse.mean():.4f} ± {df.rmse.std():.4f}")
print(f"MAE  : {df.mae.mean():.4f} ± {df.mae.std():.4f}")
print(f"예측 속도 : {df.avg_ms.mean():.2f} ms (± {df.avg_ms.std():.2f})")

print("\n=== 페르소나 추천 결과 (Seed 10) ===")
print(f"헤비 유저 (userId={PERSONA_HEAVY_USER})")
for x in best_reco['heavy']:
    print(f"  → {x['title']}  (예측 평점 {x['predicted_rating']})")

print(f"\n드라마 전문가 (userId={PERSONA_GENRE_SPECIALIST})")
for x in best_reco['specialist']:
    print(f"  → {x['title']}  (예측 평점 {x['predicted_rating']})")

   seed    rmse     mae  avg_ms
0    10  0.8492  0.6454  5.2448
1    21  0.8651  0.6592  3.0419
2    35  0.8629  0.6629  3.6564
3    42  0.8781  0.6685  2.4253
4    57  0.8392  0.6403  2.3270

=== 평균 ± 표준편차 ===
RMSE : 0.8589 ± 0.0150
MAE  : 0.6553 ± 0.0119
예측 속도 : 3.34 ms (± 1.19)

=== 페르소나 추천 결과 (Seed 10) ===
헤비 유저 (userId=414)
  → Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)  (예측 평점 4.149)
  → Terminator 2: Judgment Day (1991)  (예측 평점 4.108)
  → Goodfellas (1990)  (예측 평점 4.097)

드라마 전문가 (userId=85)
  → Lassie (1994)  (예측 평점 5.0)
  → Cats Don't Dance (1997)  (예측 평점 5.0)
  → Barney's Great Adventure (1998)  (예측 평점 5.0)
